In [1]:
import csv
import tarfile
import pandas as pd
from pathlib import Path

In [5]:
data = pd.read_csv("data/data_good.csv")
total = pd.read_csv("data/Total_old.csv", names=["gsm", "status"])

In [6]:
data.head()

,UNIT_ID,UnitName,EntrezID,GeneSymbol,GSM441627,GSM441628,GSM441629,GSM441643,GSM441644,GSM441657,...,GSM79360,GSM79353,GSM79153,GSM79349,GSM79355,GSM79181,GSM79323,GSM79214,GSM79350,GSM79211
0,25,AFFX-HUMISGF3A/M97935_MA_at,6772,STAT1,258.604,271.703,143.8760,96.2013,211.158,245.954,...,260.148,103.5360,174.966,191.4550,121.6530,122.4570,129.9380,178.902,251.261,103.5830
1,26,AFFX-HUMISGF3A/M97935_MB_at,6772,STAT1,114.433,148.662,81.2455,58.1097,135.255,169.862,...,214.424,45.7677,104.428,78.0305,58.6785,65.1835,76.1533,125.455,144.751,57.6514
2,27,AFFX-HUMISGF3A/M97935_3_at,6772,STAT1,609.059,562.069,349.9580,191.2970,879.009,771.731,...,1205.720,143.9720,789.292,1128.1100,287.0620,334.0280,386.8000,652.239,1164.150,269.3320
3,31,AFFX-HUMGAPDH/M33197_5_at,2597,GAPDH,4470.210,4570.150,2186.3100,4799.3800,6296.870,4864.770,...,2647.440,2678.1600,1562.820,1009.7900,4489.6300,2030.0600,2990.5000,2602.680,1699.050,2332.6800
4,32,AFFX-HUMGAPDH/M33197_M_at,2597,GAPDH,5183.690,4677.370,2771.9800,5587.2800,7326.030,5456.950,...,3957.630,4781.1200,2632.850,2252.8800,6344.9700,3244.9100,4166.8100,3619.780,2916.770,4011.9500


In [7]:
total.head()

,gsm,status
0,GSM441627,TooLate0
1,GSM441628,relapse
2,GSM441629,relapse
3,GSM441643,relapse
4,GSM441644,relapse


In [8]:
total["status"].unique()

array(['TooLate0', 'relapse', 'no_relapse', 'TooEarly0', 'TooLate1',
       'test1relapse', 'TooEarly1', 'test1no_relapse', 'test2relapse',
       'TooLate2', 'test2no_relapse', 'TooEarly2', 'NewTest1_Relapse',
       'NewTest1_NoRelapse', 'TooLate3', 'TooEarly3', 'NewTest2_Relapse',
       'NewTest2_NoRelapse', 'TooLate4', 'TooEarly4'], dtype=object)

In [25]:
def read_data():
    data = pd.read_csv("data/data_good.csv")
    total = pd.read_csv("data/Total_old.csv", names=["gsm", "status"])
    return data, total

def drop_grey_cols(data, total):
    status_list =['relapse', 'no_relapse', 'test1relapse',
              'test1no_relapse', 'test2relapse',
              'test2no_relapse', 'NewTest1_Relapse',
              'NewTest1_NoRelapse', 'NewTest2_Relapse', 'NewTest2_NoRelapse']
    update_gsm = total[total.status.isin(status_list)].gsm
    new_cols = pd.Series(["GeneSymbol"]).append(update_gsm)

    # Drop grey status
    total = total[total.gsm.isin(update_gsm)]
    data = data.filter(items=new_cols)
    return data, total


In [26]:
data, total = read_data()
data, total = drop_grey_cols(data, total)
total

,gsm,status
1,GSM441628,relapse
2,GSM441629,relapse
3,GSM441643,relapse
4,GSM441644,relapse
5,GSM441657,relapse
...,...,...
995,GSM79256,NewTest2_NoRelapse
996,GSM79307,NewTest2_NoRelapse
997,GSM79194,NewTest2_NoRelapse
998,GSM79179,NewTest2_NoRelapse


In [27]:
data


,GeneSymbol,GSM441628,GSM441629,GSM441643,GSM441644,GSM441657,GSM441663,GSM441672,GSM441677,GSM441689,...,GSM79316,GSM79301,GSM79303,GSM79278,GSM79158,GSM79256,GSM79307,GSM79194,GSM79179,GSM79182
0,STAT1,271.7030,143.8760,96.2013,211.1580,245.9540,227.1580,369.5200,289.5180,137.6400,...,155.0600,139.5640,173.0080,183.1150,159.9650,122.8550,134.6120,157.5450,129.4740,124.1000
1,STAT1,148.6620,81.2455,58.1097,135.2550,169.8620,148.2000,190.4340,144.6220,88.7613,...,85.0733,86.2349,104.7460,60.2855,95.5286,58.8385,66.8661,74.5329,68.6508,69.4676
2,STAT1,562.0690,349.9580,191.2970,879.0090,771.7310,1017.0800,985.2080,730.3540,435.2680,...,485.3350,335.5130,686.7610,287.5500,754.0570,264.5480,246.7240,195.2160,245.2250,440.0420
3,GAPDH,4570.1500,2186.3100,4799.3800,6296.8700,4864.7700,3227.5400,4956.9400,4993.3600,3509.2400,...,2632.7900,2159.2200,3398.4900,1108.8800,1744.3200,1778.3400,1707.9800,1045.2000,2752.3600,1839.2100
4,GAPDH,4677.3700,2771.9800,5587.2800,7326.0300,5456.9500,3495.8900,5479.1200,4882.8900,3785.1700,...,3625.7000,3060.1500,5238.4500,2016.6800,3021.6800,2409.5600,2181.3000,2785.2900,3935.0600,2991.3200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15549,NAA40,54.4646,50.2889,50.6439,50.2440,45.1267,36.5843,42.9146,67.8202,74.0205,...,59.2511,59.7173,82.1648,41.8699,69.3741,58.1110,78.4802,55.5876,42.6167,57.9080
15550,BTRC,52.5145,43.5372,64.3767,48.1749,41.5742,46.1108,57.5067,41.9006,45.1893,...,68.8718,49.3107,61.5488,68.4528,51.3720,62.1168,55.3434,70.1898,62.0527,57.5067
15551,TBX10,47.5438,43.9503,56.3981,57.5737,92.4918,46.7302,49.7785,40.8507,44.4658,...,42.9858,40.1002,35.8531,42.1160,40.3773,41.5986,42.0447,55.3962,46.3305,41.0901
15552,KCNE4,32.1067,356.4300,244.8000,50.3291,126.3150,114.7860,336.0880,33.4743,62.8083,...,49.9980,230.9910,45.8824,53.5128,105.7030,56.9028,90.1375,163.1580,179.0010,46.0703
